In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Data Preprocessing

In [4]:
import os

In [11]:
files = os.listdir("./data")
json_files = [file for file in files if file.endswith(".jsonl")]

for file in json_files:
    task = file.split(".")[0]
    with open(f"data/{file}", "r", encoding='utf-8') as f:
        data = [json.loads(line) for line in f.readlines()]

# Zero-shot llama3.2-1b

In [24]:
model_id = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [31]:
def question_from_row(row):
    prompt = f"""
    ### INSTRUCTION: ANSWER WITH A, B, C, OR D ONLY.
    ### STORY: {row['STORY']}
    ### QUESTION: {row['QUESTION']}
    ### A. {row['OPTION-A']}
    ### B. {row['OPTION-B']}
    ### C. {row['OPTION-C']}
    ### D. {row['OPTION-D']}
    ### ANSWER:"""
    return prompt

In [32]:
row = data[0]
prompt = question_from_row(row)
print("Prompt:\n", prompt)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

with torch.no_grad():
    final_logits = model(input_ids).logits[0,-1]
answer_idxs = torch.tensor([tokenizer.encode(l)[-1] for l in 'ABCD'])
answer_logits = final_logits[answer_idxs]
answer_probs = final_logits.softmax(dim=0)[answer_idxs]

print('Answer probs (all):', answer_probs)
print('Predicted answer:', "ABCD"[answer_probs.argmax()])
print('True answer:', row['答案\nANSWER'])

Prompt:
 
    ### INSTRUCTION: ANSWER WITH A, B, C, OR D ONLY.
    ### STORY: Almost every letter to Laura Company contains a check. Today, Laura receives 5 letters.  Laura tells you on the phone "I look at 3 out of 5 letters. There are checks in 2 of the letters.
    ### QUESTION: Before Laura calls you, how many of these 5 letters do you think contain checks?
    ### A. Before the call, 0 letters contain checks.
    ### B. Before the call, 1 letter contains a check.
    ### C. Before the call, 2 letters contain checks.
    ### D. Before the call, 4 letters contain checks.
    ### ANSWER:
Answer probs (all): tensor([0.0002, 0.0005, 0.0004, 0.0002])
Predicted answer: B
True answer: D
